In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd
import csv
import json
import logging
from logging import Logger
import os
import sys
from typing import Callable, Dict, List, Tuple, Literal, Union
from functools import reduce
from copy import deepcopy

from rdkit import Chem

from tensorboardX import SummaryWriter
import torch
import torch.nn as nn
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler, ExponentialLR
from tqdm import tqdm, trange

import chemprop
from chemprop.args import TrainArgs, PredictArgs, HyperoptArgs
from chemprop.data import get_task_names, get_data, validate_dataset_type, MoleculeDataset, MoleculeDataLoader
from chemprop.train import run_training, train, predict
from chemprop.constants import TEST_SCORES_FILE_NAME, TRAIN_LOGGER_NAME, MODEL_FILE_NAME, HYPEROPT_LOGGER_NAME
from chemprop.utils import create_logger, makedirs, load_args, update_prediction_args, load_checkpoint, load_scalers

## Training Args

In [2]:
args = TrainArgs()

# Common
args.smiles_columns: List[str] = ['SMILES']
args.gpu: int = 1
args.features_path: List[str] = ["../../Data/ML_data/Data_split/Features/RN_train.npz"]
args.no_features_scaling: bool = True

# # TrainArg
args.data_path: str = "../../Data/ML_data/Data_split/train.csv"
args.target_columns: List[str] = ['G1','G2','G3','G4','G5','G6','G7','G8','G9','G10','G11','G12','G13']
args.dataset_type: Literal['regression', 'classification', 'multiclass'] = 'classification'
args.separate_val_path: str = "../../Data/ML_data/Data_split/val.csv"
args.separate_test_path: str = "../../Data/ML_data/Data_split/test.csv"
args.save_dir: str = '../../Results/Trained_model/DMPNN_RN_Ensemble_5'

# # Model arguments
args.separate_val_features_path: List[str] = ["../../Data/ML_data/Data_split/Features/RN_val.npz"]
args.separate_test_features_path: List[str] = ["../../Data/ML_data/Data_split/Features/RN_test.npz"]
args.config_path: str = "../../Data/ML_data/Data_split/Opt_hyperpars/DMPNN_RN.json"
args.ensemble_size: int = 5

args.process_args()

## Load data

In [3]:
# logger: quite and verbose file
logger = create_logger(name=TRAIN_LOGGER_NAME, save_dir=args.save_dir, quiet=args.quiet)
if logger is not None:
    debug, info = logger.debug, logger.info
else:
    debug = info = print

# Initialize relevant variables
init_seed = args.seed
save_dir = args.save_dir
args.task_names = get_task_names(path=args.data_path, smiles_columns=args.smiles_columns,
                                 target_columns=args.target_columns, ignore_columns=args.ignore_columns)


# Print args
# debug('Args')
# debug({key:value for key, value in args.__dict__.items() if not key.startswith('_') and not callable(key)})

# Save args
makedirs(args.save_dir)
#     args.save(os.path.join(args.save_dir, 'args.json'))

# Get data
debug('Loading data')
data = get_data(
    path=args.data_path,
    args=args,
    smiles_columns=args.smiles_columns,
    logger=logger,
    skip_none_targets=True
)
validate_dataset_type(data, dataset_type=args.dataset_type)
args.features_size = data.features_size()

if args.atom_descriptors == 'descriptor':
    args.atom_descriptors_size = data.atom_descriptors_size()
    args.ffn_hidden_size += args.atom_descriptors_size
elif args.atom_descriptors == 'feature':
    args.atom_features_size = data.atom_features_size()
    set_extra_atom_fdim(args.atom_features_size)
if args.bond_features_path is not None:
    args.bond_features_size = data.bond_features_size()
    set_extra_bond_fdim(args.bond_features_size)

debug(f'Number of tasks = {args.num_tasks}')

Loading data
37773it [00:00, 138056.15it/s]
100%|██████████| 37773/37773 [00:04<00:00, 8348.60it/s] 
Number of tasks = 13


## Training

In [4]:
# chemprop cross_validate
# Run training on different random seeds for each fold
all_scores = defaultdict(list)
for fold_num in range(args.num_folds):
    info(f'Fold {fold_num}')
    args.seed = init_seed + fold_num
    args.save_dir = os.path.join(save_dir, f'fold_{fold_num}')
    makedirs(args.save_dir)
    data.reset_features_and_targets()
    model_scores = run_training(args, data, logger)
    for metric, scores in model_scores.items():
        all_scores[metric].append(scores)
all_scores = dict(all_scores)    

Fold 0
Splitting data with seed 0
4723it [00:00, 186670.85it/s]
100%|██████████| 4723/4723 [00:00<00:00, 8216.36it/s]
4721it [00:00, 186530.29it/s]
100%|██████████| 4721/4721 [00:00<00:00, 8311.52it/s]
Class sizes
G1 0: 96.70%, 1: 3.30%
G2 0: 97.37%, 1: 2.63%
G3 0: 96.61%, 1: 3.39%
G4 0: 95.81%, 1: 4.19%
G5 0: 95.92%, 1: 4.08%
G6 0: 96.50%, 1: 3.50%
G7 0: 95.89%, 1: 4.11%
G8 0: 96.01%, 1: 3.99%
G9 0: 96.67%, 1: 3.33%
G10 0: 96.67%, 1: 3.33%
G11 0: 93.72%, 1: 6.28%
G12 0: 96.58%, 1: 3.42%
G13 0: 94.79%, 1: 5.21%
Total size = 37,773 | train size = 37,773 | val size = 4,721 | test size = 4,723
Building model 0
MoleculeModel(
  (sigmoid): Sigmoid()
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout_layer): Dropout(p=0.35000000000000003, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=1100, bias=False)
        (W_h): Linear(in_features=1100, out_features=1100, bias=False)
        (W_o): Linear(in_features=1233, 

  1%|          | 4/756 [00:00<01:07, 11.21it/s]Loss = 1.8982e-01, PNorm = 51.1379, GNorm = 0.1752, lr_0 = 5.5417e-04

  2%|▏         | 12/756 [00:00<00:30, 24.20it/s]Loss = 1.8663e-01, PNorm = 51.1419, GNorm = 0.0992, lr_0 = 5.6013e-04

  3%|▎         | 20/756 [00:00<00:24, 30.43it/s]Loss = 1.2350e-01, PNorm = 51.1587, GNorm = 0.1133, lr_0 = 5.6609e-04

  4%|▍         | 34/756 [00:01<00:20, 34.98it/s]Loss = 1.6582e-01, PNorm = 51.1555, GNorm = 0.1259, lr_0 = 5.7205e-04

  6%|▌         | 42/756 [00:01<00:19, 36.21it/s]Loss = 1.4670e-01, PNorm = 51.1698, GNorm = 0.0888, lr_0 = 5.7801e-04

  7%|▋         | 54/756 [00:01<00:19, 36.92it/s]Loss = 1.6398e-01, PNorm = 51.1872, GNorm = 0.1154, lr_0 = 5.8397e-04

  8%|▊         | 62/756 [00:01<00:18, 36.93it/s]Loss = 1.4738e-01, PNorm = 51.2022, GNorm = 0.0980, lr_0 = 5.8993e-04

 10%|▉         | 74/756 [00:02<00:18, 37.05it/s]Loss = 1.4568e-01, PNorm = 51.2123, GNorm = 0.1047, lr_0 = 5.9589e-04

 11%|█         | 82/756 [00:02<00:18, 37.24it/s]L

  1%|          | 9/756 [00:00<00:36, 20.64it/s]Loss = 1.7451e-01, PNorm = 53.2559, GNorm = 0.0996, lr_0 = 9.9859e-04

  2%|▏         | 17/756 [00:00<00:25, 28.71it/s]Loss = 1.8309e-01, PNorm = 53.2886, GNorm = 0.1104, lr_0 = 9.9750e-04

  4%|▍         | 29/756 [00:01<00:20, 34.72it/s]Loss = 1.2966e-01, PNorm = 53.3583, GNorm = 0.0750, lr_0 = 9.9641e-04

  5%|▍         | 37/756 [00:01<00:19, 36.19it/s]Loss = 1.3311e-01, PNorm = 53.3893, GNorm = 0.0897, lr_0 = 9.9533e-04

  6%|▋         | 49/756 [00:01<00:18, 37.26it/s]Loss = 1.4696e-01, PNorm = 53.4260, GNorm = 0.1306, lr_0 = 9.9424e-04

  8%|▊         | 57/756 [00:01<00:18, 37.47it/s]Loss = 1.7943e-01, PNorm = 53.4355, GNorm = 0.1243, lr_0 = 9.9316e-04

  9%|▉         | 69/756 [00:02<00:18, 37.77it/s]Loss = 1.4104e-01, PNorm = 53.4915, GNorm = 0.0760, lr_0 = 9.9208e-04

 10%|█         | 77/756 [00:02<00:17, 37.74it/s]Loss = 2.0371e-01, PNorm = 53.5104, GNorm = 0.0921, lr_0 = 9.9100e-04

 12%|█▏        | 89/756 [00:02<00:17, 37.33it/s]L


 55%|█████▌    | 417/756 [00:11<00:09, 34.90it/s]Loss = 1.7504e-01, PNorm = 55.0915, GNorm = 0.0799, lr_0 = 9.5497e-04

 57%|█████▋    | 429/756 [00:12<00:09, 36.02it/s]Loss = 1.5678e-01, PNorm = 55.1858, GNorm = 0.0908, lr_0 = 9.5393e-04

 58%|█████▊    | 437/756 [00:12<00:08, 36.24it/s]Loss = 1.6297e-01, PNorm = 55.2474, GNorm = 0.0875, lr_0 = 9.5289e-04

 59%|█████▉    | 449/756 [00:12<00:08, 36.52it/s]Loss = 1.3256e-01, PNorm = 55.3183, GNorm = 0.1758, lr_0 = 9.5186e-04

 60%|██████    | 457/756 [00:12<00:08, 37.17it/s]Loss = 1.5904e-01, PNorm = 55.3340, GNorm = 0.1346, lr_0 = 9.5082e-04

 62%|██████▏   | 469/756 [00:13<00:07, 37.45it/s]Loss = 1.6176e-01, PNorm = 55.3941, GNorm = 0.0900, lr_0 = 9.4979e-04

 63%|██████▎   | 477/756 [00:13<00:07, 37.56it/s]Loss = 1.7443e-01, PNorm = 55.4157, GNorm = 0.0869, lr_0 = 9.4875e-04

 65%|██████▍   | 489/756 [00:13<00:07, 37.00it/s]Loss = 1.1839e-01, PNorm = 55.4685, GNorm = 0.0821, lr_0 = 9.4772e-04

 66%|██████▌   | 497/756 [00:14<00:07, 


 61%|██████    | 459/756 [00:14<00:08, 34.55it/s]Loss = 1.7157e-01, PNorm = 58.5143, GNorm = 0.1126, lr_0 = 8.7528e-04

 63%|██████▎   | 473/756 [00:14<00:09, 29.52it/s]Loss = 1.7897e-01, PNorm = 58.5581, GNorm = 0.1212, lr_0 = 8.7433e-04

 64%|██████▍   | 482/756 [00:15<00:10, 26.41it/s]Loss = 1.4035e-01, PNorm = 58.6024, GNorm = 0.1001, lr_0 = 8.7338e-04

 65%|██████▍   | 490/756 [00:15<00:09, 29.39it/s]Loss = 1.8138e-01, PNorm = 58.6117, GNorm = 0.0973, lr_0 = 8.7243e-04

 66%|██████▌   | 499/756 [00:15<00:08, 31.87it/s]Loss = 1.2849e-01, PNorm = 58.6689, GNorm = 0.0680, lr_0 = 8.7148e-04

 68%|██████▊   | 513/756 [00:16<00:07, 32.50it/s]Loss = 1.3699e-01, PNorm = 58.7150, GNorm = 0.0927, lr_0 = 8.7053e-04

 69%|██████▉   | 522/756 [00:16<00:07, 31.98it/s]Loss = 1.6532e-01, PNorm = 58.7454, GNorm = 0.1040, lr_0 = 8.6958e-04

 70%|███████   | 530/756 [00:16<00:06, 32.87it/s]Loss = 2.1792e-01, PNorm = 58.7759, GNorm = 0.1212, lr_0 = 8.6863e-04

 71%|███████▏  | 539/756 [00:16<00:06, 

...

 67%|██████▋   | 505/756 [00:16<00:07, 31.55it/s]Loss = 9.8140e-02, PNorm = 95.7438, GNorm = 0.4795, lr_0 = 1.3107e-04

 68%|██████▊   | 517/756 [00:16<00:07, 33.03it/s]Loss = 9.1948e-02, PNorm = 95.7484, GNorm = 0.2992, lr_0 = 1.3093e-04

 69%|██████▉   | 525/756 [00:16<00:07, 32.22it/s]Loss = 1.2385e-01, PNorm = 95.7499, GNorm = 0.3719, lr_0 = 1.3078e-04

 71%|███████   | 537/756 [00:17<00:07, 30.59it/s]Loss = 1.1653e-01, PNorm = 95.7528, GNorm = 0.2726, lr_0 = 1.3064e-04

 72%|███████▏  | 545/756 [00:17<00:06, 30.66it/s]Loss = 1.0669e-01, PNorm = 95.7571, GNorm = 0.2742, lr_0 = 1.3050e-04

 74%|███████▎  | 557/756 [00:17<00:07, 27.45it/s]Loss = 1.3272e-01, PNorm = 95.7598, GNorm = 0.2625, lr_0 = 1.3036e-04

 75%|███████▌  | 568/756 [00:18<00:06, 29.21it/s]Loss = 1.1760e-01, PNorm = 95.7645, GNorm = 0.3431, lr_0 = 1.3022e-04

 76%|███████▌  | 576/756 [00:18<00:05, 30.04it/s]Loss = 1.0771e-01, PNorm = 95.7684, GNorm = 0.2996, lr_0 = 1.3007e-04

 78%|███████▊  | 588/756 [00:18<00:05, 3


 69%|██████▊   | 519/756 [00:14<00:06, 35.78it/s]Loss = 1.2986e-01, PNorm = 96.0351, GNorm = 0.3184, lr_0 = 1.2053e-04

 70%|███████   | 531/756 [00:15<00:06, 35.67it/s]Loss = 1.7684e-01, PNorm = 96.0370, GNorm = 0.5842, lr_0 = 1.2039e-04

 71%|███████▏  | 539/756 [00:15<00:06, 35.18it/s]Loss = 1.4673e-01, PNorm = 96.0422, GNorm = 0.2787, lr_0 = 1.2026e-04

 73%|███████▎  | 551/756 [00:15<00:05, 35.53it/s]Loss = 1.0610e-01, PNorm = 96.0491, GNorm = 0.2249, lr_0 = 1.2013e-04

 74%|███████▍  | 559/756 [00:16<00:05, 35.04it/s]Loss = 1.0487e-01, PNorm = 96.0574, GNorm = 0.5336, lr_0 = 1.2000e-04

 76%|███████▌  | 571/756 [00:16<00:05, 34.12it/s]Loss = 1.1668e-01, PNorm = 96.0616, GNorm = 0.3876, lr_0 = 1.1987e-04

 77%|███████▋  | 579/756 [00:16<00:05, 33.64it/s]Loss = 1.1713e-01, PNorm = 96.0631, GNorm = 0.4484, lr_0 = 1.1974e-04

 78%|███████▊  | 591/756 [00:17<00:05, 32.79it/s]Loss = 1.0999e-01, PNorm = 96.0658, GNorm = 0.4023, lr_0 = 1.1961e-04

 79%|███████▉  | 599/756 [00:17<00:04, 


 16%|█▋        | 124/756 [00:04<00:18, 33.67it/s]Loss = 1.4846e-01, PNorm = 96.1751, GNorm = 0.4183, lr_0 = 1.1588e-04

 18%|█▊        | 136/756 [00:04<00:17, 35.03it/s]Loss = 1.6244e-01, PNorm = 96.1757, GNorm = 0.4222, lr_0 = 1.1575e-04

 19%|█▉        | 144/756 [00:04<00:17, 35.32it/s]Loss = 1.4452e-01, PNorm = 96.1810, GNorm = 0.4981, lr_0 = 1.1563e-04

 21%|██        | 156/756 [00:04<00:16, 35.98it/s]Loss = 1.1962e-01, PNorm = 96.1860, GNorm = 0.2385, lr_0 = 1.1550e-04

 22%|██▏       | 164/756 [00:05<00:16, 36.26it/s]Loss = 1.1735e-01, PNorm = 96.1897, GNorm = 0.3814, lr_0 = 1.1537e-04

 23%|██▎       | 176/756 [00:05<00:16, 34.72it/s]Loss = 1.4111e-01, PNorm = 96.1929, GNorm = 0.3718, lr_0 = 1.1525e-04

 24%|██▍       | 184/756 [00:05<00:16, 34.44it/s]Loss = 1.5227e-01, PNorm = 96.1943, GNorm = 0.4749, lr_0 = 1.1512e-04

 26%|██▌       | 196/756 [00:06<00:16, 34.61it/s]Loss = 1.3877e-01, PNorm = 96.1943, GNorm = 0.4658, lr_0 = 1.1500e-04

 27%|██▋       | 204/756 [00:06<00:15, 

 17%|█▋        | 130/756 [00:04<00:20, 29.86it/s]Loss = 1.1964e-01, PNorm = 96.4262, GNorm = 0.3048, lr_0 = 1.0667e-04

 18%|█▊        | 138/756 [00:05<00:20, 30.67it/s]Loss = 1.0725e-01, PNorm = 96.4296, GNorm = 0.3483, lr_0 = 1.0656e-04

 20%|█▉        | 150/756 [00:05<00:22, 27.15it/s]Loss = 1.0120e-01, PNorm = 96.4327, GNorm = 0.3574, lr_0 = 1.0644e-04

 21%|██▏       | 161/756 [00:05<00:19, 29.98it/s]Loss = 1.1364e-01, PNorm = 96.4364, GNorm = 0.2556, lr_0 = 1.0632e-04

 22%|██▏       | 169/756 [00:06<00:19, 30.76it/s]Loss = 1.0481e-01, PNorm = 96.4404, GNorm = 0.6229, lr_0 = 1.0621e-04

 24%|██▍       | 181/756 [00:06<00:20, 27.99it/s]Loss = 1.2955e-01, PNorm = 96.4441, GNorm = 0.4012, lr_0 = 1.0609e-04

 25%|██▌       | 190/756 [00:06<00:20, 28.09it/s]Loss = 1.0054e-01, PNorm = 96.4474, GNorm = 0.1310, lr_0 = 1.0598e-04

 26%|██▌       | 198/756 [00:07<00:19, 28.18it/s]Loss = 1.2699e-01, PNorm = 96.4519, GNorm = 0.5388, lr_0 = 1.0586e-04

 28%|██▊       | 210/756 [00:07<00:17, 3


 72%|███████▏  | 541/756 [00:18<00:08, 26.30it/s]Loss = 1.1733e-01, PNorm = 96.5509, GNorm = 1.0859, lr_0 = 1.0201e-04

 73%|███████▎  | 549/756 [00:18<00:07, 26.24it/s]Loss = 1.1862e-01, PNorm = 96.5533, GNorm = 0.4071, lr_0 = 1.0190e-04

 74%|███████▍  | 561/756 [00:18<00:06, 30.76it/s]Loss = 1.1694e-01, PNorm = 96.5561, GNorm = 0.4934, lr_0 = 1.0179e-04

 75%|███████▌  | 569/756 [00:19<00:05, 31.28it/s]Loss = 1.2216e-01, PNorm = 96.5574, GNorm = 0.2832, lr_0 = 1.0168e-04

 76%|███████▋  | 578/756 [00:19<00:05, 32.19it/s]Loss = 1.0474e-01, PNorm = 96.5607, GNorm = 0.3415, lr_0 = 1.0157e-04

 78%|███████▊  | 590/756 [00:19<00:05, 31.13it/s]Loss = 1.0231e-01, PNorm = 96.5638, GNorm = 0.4490, lr_0 = 1.0146e-04

 79%|███████▉  | 598/756 [00:20<00:05, 31.11it/s]Loss = 1.1965e-01, PNorm = 96.5652, GNorm = 0.4607, lr_0 = 1.0135e-04

 81%|████████  | 610/756 [00:20<00:04, 31.76it/s]Loss = 1.4979e-01, PNorm = 96.5659, GNorm = 0.3315, lr_0 = 1.0124e-04

 82%|████████▏ | 618/756 [00:20<00:04, 

## Saving

In [5]:
model_scores

{'auc': [0.8255631355386794,
  0.7938604681638737,
  0.8339526113660627,
  0.8218594121079144,
  0.8066843968627716,
  0.8250288579511217,
  0.7831200284121299,
  0.8304504041510574,
  0.8582291937058875,
  0.8375058180281298,
  0.7720368811074656,
  0.8395092770865064,
  0.8156454894867737]}

In [6]:
# Convert scores to numpy arrays
for metric, scores in all_scores.items():
    all_scores[metric] = np.array(scores)

# Report results
info(f'{args.num_folds}-fold cross validation')

# Report scores for each fold
for fold_num in range(args.num_folds):
    for metric, scores in all_scores.items():
        info(f'\tSeed {init_seed + fold_num} ==> test {metric} = {np.nanmean(scores[fold_num]):.6f}')

        if args.show_individual_scores:
            for task_name, score in zip(args.task_names, scores[fold_num]):
                info(f'\t\tSeed {init_seed + fold_num} ==> test {task_name} {metric} = {score:.6f}')

# Report scores across folds
for metric, scores in all_scores.items():
    avg_scores = np.nanmean(scores, axis=1)  # average score for each model across tasks
    mean_score, std_score = np.nanmean(avg_scores), np.nanstd(avg_scores)
    info(f'Overall test {metric} = {mean_score:.6f} +/- {std_score:.6f}')

    if args.show_individual_scores:
        for task_num, task_name in enumerate(args.task_names):
            info(f'\tOverall test {task_name} {metric} = '
                 f'{np.nanmean(scores[:, task_num]):.6f} +/- {np.nanstd(scores[:, task_num]):.6f}')

# Save scores
with open(os.path.join(save_dir, TEST_SCORES_FILE_NAME), 'w') as f:
    writer = csv.writer(f)

    header = ['Task']
    for metric in args.metrics:
        header += [f'Mean {metric}', f'Standard deviation {metric}'] + \
                  [f'Fold {i} {metric}' for i in range(args.num_folds)]
    writer.writerow(header)

    for task_num, task_name in enumerate(args.task_names):
        row = [task_name]
        for metric, scores in all_scores.items():
            task_scores = scores[:, task_num]
            mean, std = np.nanmean(task_scores), np.nanstd(task_scores)
            row += [mean, std] + task_scores.tolist()
        writer.writerow(row)

# Determine mean and std score of main metric
avg_scores = np.nanmean(all_scores[args.metric], axis=1)
mean_score, std_score = np.nanmean(avg_scores), np.nanstd(avg_scores)
print(mean_score, std_score)

1-fold cross validation
	Seed 0 ==> test auc = 0.818727
Overall test auc = 0.818727 +/- 0.000000


0.8187266133821824 0.0


# Make prediction

In [7]:
args = PredictArgs()

args.smiles_columns: List[str] = ['SMILES']   
args.gpu: int = 1
args.checkpoint_dir = "../../Results/Trained_model/DMPNN_RN_Ensemble_5/fold_0/"
args.test_path: str = "../../Data/Mtb_inhibitors/Mtb_inhibitors.csv"
args.features_path: List[str] = ["../../Data/Mtb_inhibitors/Features/RN_Mtb_inhibitors.npz"]
args.preds_path: str = "../../Results/Mtb_inhibitors_pred/Mtb_inhibitors_DMPNN_preds.csv"
args.no_features_scaling = True

args.process_args()

In [8]:
# chemprop make_predictions

print('Loading training args')
train_args = load_args(args.checkpoint_paths[0])

num_tasks, task_names = train_args.num_tasks, train_args.task_names

update_prediction_args(predict_args=args, train_args=train_args)
args: Union[PredictArgs, TrainArgs]

if args.atom_descriptors == 'feature':
    set_extra_atom_fdim(train_args.atom_features_size)

if args.bond_features_path is not None:
    set_extra_bond_fdim(train_args.bond_features_size)

print('Loading data')
full_data = get_data(path=args.test_path, smiles_columns=args.smiles_columns, target_columns=[], ignore_columns=[],
                         skip_invalid_smiles=False, args=args, store_row=not args.drop_extra_columns)

print('Validating SMILES')
full_to_valid_indices = {}
valid_index = 0
for full_index in range(len(full_data)):
    if all(mol is not None for mol in full_data[full_index].mol):
        full_to_valid_indices[full_index] = valid_index
        valid_index += 1

test_data = MoleculeDataset([full_data[i] for i in sorted(full_to_valid_indices.keys())])

print(f'Test size = {len(test_data):,}')

# Predict with each model individually and sum predictions
if args.dataset_type == 'multiclass':
    sum_preds = np.zeros((len(test_data), num_tasks, args.multiclass_num_classes))
else:
    sum_preds = np.zeros((len(test_data), num_tasks))

# Create data loader
test_data_loader = MoleculeDataLoader(
    dataset=test_data,
    batch_size=args.batch_size,
    num_workers=args.num_workers
)

# Partial results for variance robust calculation.
if args.ensemble_variance:
    all_preds = np.zeros((len(test_data), num_tasks, len(args.checkpoint_paths)))

print(f'Predicting with an ensemble of {len(args.checkpoint_paths)} models')
for index, checkpoint_path in enumerate(tqdm(args.checkpoint_paths, total=len(args.checkpoint_paths))):
    # Load model and scalers
    model = load_checkpoint(checkpoint_path, device=args.device)
    scaler, features_scaler, atom_descriptor_scaler, bond_feature_scaler = load_scalers(checkpoint_path)

    # Normalize features
    if args.features_scaling or train_args.atom_descriptor_scaling or train_args.bond_feature_scaling:
        test_data.reset_features_and_targets()
        if args.features_scaling:
            test_data.normalize_features(features_scaler)
        if train_args.atom_descriptor_scaling and args.atom_descriptors is not None:
            test_data.normalize_features(atom_descriptor_scaler, scale_atom_descriptors=True)
        if train_args.bond_feature_scaling and args.bond_features_size > 0:
            test_data.normalize_features(bond_feature_scaler, scale_bond_features=True)

    # Make predictions
    model_preds = predict(
        model=model,
        data_loader=test_data_loader,
        scaler=scaler
    )
    sum_preds += np.array(model_preds)
    if args.ensemble_variance:
        all_preds[:, :, index] = model_preds

# Ensemble predictions
avg_preds = sum_preds / len(args.checkpoint_paths)
avg_preds = avg_preds.tolist()

if args.ensemble_variance:
    all_epi_uncs = np.var(all_preds, axis=2)
    all_epi_uncs = all_epi_uncs.tolist()

# Save predictions
print(f'Saving predictions to {args.preds_path}')
assert len(test_data) == len(avg_preds)
if args.ensemble_variance:
    assert len(test_data) == len(all_epi_uncs)
makedirs(args.preds_path, isfile=True)

# Get prediction column names
if args.dataset_type == 'multiclass':
    task_names = [f'{name}_class_{i}' for name in task_names for i in range(args.multiclass_num_classes)]
else:
    task_names = task_names

# Copy predictions over to full_data
for full_index, datapoint in enumerate(full_data):
    valid_index = full_to_valid_indices.get(full_index, None)
    preds = avg_preds[valid_index] if valid_index is not None else ['Invalid SMILES'] * len(task_names)
    if args.ensemble_variance:
        epi_uncs = all_epi_uncs[valid_index] if valid_index is not None else ['Invalid SMILES'] * len(task_names)

    # If extra columns have been dropped, add back in SMILES columns
    if args.drop_extra_columns:
        datapoint.row = OrderedDict()

        smiles_columns = args.smiles_columns

        for column, smiles in zip(smiles_columns, datapoint.smiles):
            datapoint.row[column] = smiles

    # Add predictions columns
    if args.ensemble_variance:
        for pred_name, pred, epi_unc in zip(task_names, preds, epi_uncs):
            datapoint.row[pred_name] = pred
            datapoint.row[pred_name+'_epi_unc'] = epi_unc
    else:
        for pred_name, pred in zip(task_names, preds):
            datapoint.row[pred_name] = pred

# Save
with open(args.preds_path, 'w') as f:
    writer = csv.DictWriter(f, fieldnames=full_data[0].row.keys())
    writer.writeheader()

    for datapoint in full_data:
        writer.writerow(datapoint.row)


Loading training args


44it [00:00, 163318.03it/s]
  0%|          | 0/5 [00:00<?, ?it/s]

Loading data
Validating SMILES
Test size = 44
Predicting with an ensemble of 5 models



  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Moving model to cuda


...

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Moving model to cuda



  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Moving model to cuda



100%|██████████| 5/5 [00:02<00:00,  2.37it/s]

Saving predictions to ../../Results/Mtb_inhibitors_pred/Mtb_inhibitors_DMPNN_preds.csv
